# Notebook Setup

In [2]:
!pip install "transformers==4.31.0" "datasets[s3]==2.13.0" sagemaker --upgrade --quiet



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install huggingface_hub


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install datasets


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [5]:
!pip install --upgrade langchain==0.0.249


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [6]:
!pip install transformers


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [7]:
!pip install bs4


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [8]:
!pip install --upgrade accelerate


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
!huggingface-cli login --token ***enter huggingface token***

In [9]:
import sagemaker
import boto3
import time
import json

from datasets import Dataset
from langchain.document_loaders import WebBaseLoader
from random import randint
from itertools import chain
from functools import partial
from transformers import AutoTokenizer
from sagemaker.huggingface import HuggingFace, HuggingFaceModel
from huggingface_hub import HfFolder

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::308819823671:role/service-role/AmazonSageMaker-ExecutionRole-20221108T215813
sagemaker bucket: sagemaker-us-east-1-308819823671
sagemaker session region: us-east-1


# Data retrieval

In [12]:
loader = WebBaseLoader(["https://aws.amazon.com/blogs/aws/preview-enable-foundation-models-to-complete-tasks-with-agents-for-amazon-bedrock/", "https://aws.amazon.com/blogs/aws/aws-entity-resolution-match-and-link-related-records-from-multiple-applications-and-data-stores/", "https://aws.amazon.com/blogs/database/the-role-of-vector-datastores-in-generative-ai-applications/", "https://aws.amazon.com/blogs/big-data/introducing-the-vector-engine-for-amazon-opensearch-serverless-now-in-preview/", "https://aws.amazon.com/blogs/big-data/build-data-integration-jobs-with-ai-companion-on-aws-glue-studio-notebook-powered-by-amazon-codewhisperer/", "https://aws.amazon.com/blogs/aws/new-amazon-ec2-p5-instances-powered-by-nvidia-h100-tensor-core-gpus-for-accelerating-generative-ai-and-hpc-applications/"])

In [13]:
data = loader.load()
data

[Document(page_content='\n\n\n\n\nPreview – Enable Foundation Models to Complete Tasks With Agents for Amazon Bedrock | AWS News Blog\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n Skip to Main Content\n\n\n\n\n\nClick here to return to Amazon Web Services homepage\n\n\n\nContact Us\n Support\xa0 \nEnglish\xa0\nMy Account\xa0\n\n\n\n\n Sign In\n\n\n  Create an AWS Account \n\n\n\n\n\n\n\n\n\nProducts\nSolutions\nPricing\nDocumentation\nLearn\nPartner Network\nAWS Marketplace\nCustomer Enablement\nEvents\nExplore More \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n Close \n\n\n\nعربي\nBahasa Indonesia\nDeutsch\nEnglish\nEspañol\nFrançais\nItaliano\nPortuguês\n\n\n\n\nTiếng Việt\nTürkçe\nΡусский\nไทย\n日本語\n한국어\n中文 (简体)\n中文 (繁體)\n\n\n\n\n\n Close \n\nMy Profile\nSign out of AWS Builder ID\nAWS Management Console\nAccount Settings\nBilling & Cost Management\nSecurity Credentials\nAWS Personal Health Dashboard\n\n\n\n Close \n\nSupport C

# Data processing

In [14]:
def strip_spaces(doc):
    return {"text": doc.page_content.replace("  ", "")}

In [15]:
stripped_data = list(map(strip_spaces, data))
stripped_data

[{'text': '\n\n\n\n\nPreview – Enable Foundation Models to Complete Tasks With Agents for Amazon Bedrock | AWS News Blog\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n Skip to Main Content\n\n\n\n\n\nClick here to return to Amazon Web Services homepage\n\n\n\nContact Us\n Support\xa0 \nEnglish\xa0\nMy Account\xa0\n\n\n\n\n Sign In\n\n\nCreate an AWS Account \n\n\n\n\n\n\n\n\n\nProducts\nSolutions\nPricing\nDocumentation\nLearn\nPartner Network\nAWS Marketplace\nCustomer Enablement\nEvents\nExplore More \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n Close \n\n\n\nعربي\nBahasa Indonesia\nDeutsch\nEnglish\nEspañol\nFrançais\nItaliano\nPortuguês\n\n\n\n\nTiếng Việt\nTürkçe\nΡусский\nไทย\n日本語\n한국어\n中文 (简体)\n中文 (繁體)\n\n\n\n\n\n Close \n\nMy Profile\nSign out of AWS Builder ID\nAWS Management Console\nAccount Settings\nBilling & Cost Management\nSecurity Credentials\nAWS Personal Health Dashboard\n\n\n\n Close \n\nSupport Center\nExpert H

In [16]:
dataset = Dataset.from_list(stripped_data)
dataset

Dataset({
    features: ['text'],
    num_rows: 6
})

In [17]:
model_id = "meta-llama/Llama-2-13b-hf" # sharded weights
tokenizer = AutoTokenizer.from_pretrained(model_id,use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [18]:
# empty list to save remainder from batches to use in next batch
remainder = {"input_ids": [], "attention_mask": [], "token_type_ids": []}

def chunk(sample, chunk_length=2048):
    # define global remainder variable to save remainder from batches to use in next batch
    global remainder
    # Concatenate all texts and add remainder from previous batch
    concatenated_examples = {k: list(chain(*sample[k])) for k in sample.keys()}
    concatenated_examples = {k: remainder[k] + concatenated_examples[k] for k in concatenated_examples.keys()}
    # get total number of tokens for batch
    batch_total_length = len(concatenated_examples[list(sample.keys())[0]])

    # get max number of chunks for batch
    if batch_total_length >= chunk_length:
        batch_chunk_length = (batch_total_length // chunk_length) * chunk_length

    # Split by chunks of max_len.
    result = {
        k: [t[i : i + chunk_length] for i in range(0, batch_chunk_length, chunk_length)]
        for k, t in concatenated_examples.items()
    }
    # add remainder to global variable for next batch
    remainder = {k: concatenated_examples[k][batch_chunk_length:] for k in concatenated_examples.keys()}
    # prepare labels
    result["labels"] = result["input_ids"].copy()
    return result


# tokenize and chunk dataset
lm_dataset = dataset.map(
    lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(dataset.features)
).map(
    partial(chunk, chunk_length=4096),
    batched=True,
)


# Print total number of samples
print(f"Total number of training samples: {len(lm_dataset)}")

Total number of training samples: 8


In [19]:
def sum_dataset_arrays(dataset):
    total = 0
    for i in range(0,8):
        total = total + len(lm_dataset[i]['input_ids'])
    return total
print(f"Total number of training tokens: {sum_dataset_arrays(lm_dataset)}")

Total number of training tokens: 32768


In [ ]:
# save train_dataset to s3
training_input_path = f's3://{sess.default_bucket()}/processed/llama/genai-nyc-summit/train'
lm_dataset.save_to_disk(training_input_path)

print("uploaded data to:")
print(f"training dataset to: {training_input_path}")


# Fine-tuning

In [ ]:
# define Training Job Name
job_name = f'huggingface-qlora-{model_id.replace("/", "-")}-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'

# hyperparameters, which are passed into the training job
hyperparameters ={
  'model_id': model_id,                             # pre-trained model
  'dataset_path': '/opt/ml/input/data/training',    # path where sagemaker will save training dataset
  'epochs': 20,                                      # number of training epochs
  'per_device_train_batch_size': 2,                 # batch size for training
  'lr': 2e-4,                                       # learning rate used during training
  'hf_token': HfFolder.get_token(),                 # huggingface token to access llama 2
  'merge_weights': True,                            # wether to merge LoRA into the model (needs more memory)
}

# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'run_clm.py',      # train script
    source_dir           = 'scripts',         # directory which includes all the files needed for training
    instance_type        = 'ml.g5.4xlarge',   # instances type used for the training job
    instance_count       = 1,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    volume_size          = 300,               # the size of the EBS volume in GB
    transformers_version = '4.28',            # the transformers version used in the training job
    pytorch_version      = '2.0',             # the pytorch_version version used in the training job
    py_version           = 'py310',           # the python version used in the training job
    hyperparameters      =  hyperparameters,  # the hyperparameters passed to the training job
    environment          = { "HUGGINGFACE_HUB_CACHE": "/tmp/.cache" }, # set env variable to cache models in /tmp
)

In [ ]:
# define a data input dictonary with our uploaded s3 uris
data = {'training': training_input_path}

# starting the train job with our uploaded datasets as input
huggingface_estimator.fit(data, wait=False)

# Deployment

In [ ]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface",
  version="0.8.2"
)

# print ecr image uri
print(f"llm image uri: {llm_image}")


In [ ]:
# sagemaker config
instance_type = "ml.g5.12xlarge"
number_of_gpu = 4
health_check_timeout = 300

# TGI config
config = {
  'HF_MODEL_ID': "/opt/ml/model", # path to where sagemaker stores the model
  'SM_NUM_GPUS': json.dumps(number_of_gpu), # Number of GPU used per replica
  'MAX_INPUT_LENGTH': json.dumps(1024),  # Max length of input text
  'MAX_TOTAL_TOKENS': json.dumps(2048),  # Max length of the generation (including input text),
  # 'HF_MODEL_QUANTIZE': "bitsandbytes", # comment in to quantize
}

# create HuggingFaceModel
llm_model = HuggingFaceModel(
  role=role,
  image_uri=llm_image,
  #model_data="s3://sagemaker-us-east-1-308819823671/huggingface-qlora-llama2-13b-chat-2023--2023-08-02-08-54-16-604/output/model.tar.gz",
  model_data="s3://sagemaker-us-east-1-308819823671/huggingface-qlora-meta-llama-Llama-2-13-2023-09-01-16-39-25-384/output/model.tar.gz",
  env=config
)

In [ ]:
# Deploy model to an endpoint
# https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy
llm = llm_model.deploy(
  #endpoint_name="llama-2-13b-chat-hf-nyc-finetuned", # alternatively "llama-2-13b-hf-nyc-finetuned" 
  endpoint_name="llama-2-13b-hf-nyc-finetuned", # alternatively "llama-2-13b-hf-nyc-finetuned"  
  initial_instance_count=1,
  instance_type=instance_type,
  # volume_size=400, # If using an instance with local SSD storage, volume_size must be None, e.g. p4 but not p3
  container_startup_health_check_timeout=health_check_timeout, # 10 minutes to be able to load the model
)


# Inference

In [33]:
basic_endpoint = 'jumpstart-dft-meta-textgeneration-llama-2-13b' 
basic_endpoint_ft = 'llama-2-13b-hf-nyc-finetuned'
chat_endpoint = 'jumpstart-dft-meta-textgeneration-llama-2-13b-f'
chat_endpoint_ft = 'llama-2-13b-chat-hf-nyc-finetuned'


In [34]:
def query_endpoint(payload, endpoint_name):
    client = boto3.client("sagemaker-runtime")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/json",
        Body=json.dumps(payload),
        CustomAttributes="accept_eula=true",
    )
    response = response["Body"].read().decode("utf8")
    response = json.loads(response)
    return response

# Base vs. chat model
## LLaMA2 

In [35]:
prompt = "Hi! I'm Aris and I am wondering what I should do today in sunny Athens."
print(f'{basic_endpoint}: {query_endpoint({"inputs": prompt, "parameters": {"max_new_tokens": 200, "top_p": 0.9, "temperature": 0.01, "return_full_text": False}}, basic_endpoint)}')

jumpstart-dft-meta-textgeneration-llama-2-13b: [{'generation': " I'm a bit bored and I'm looking for something to do. I'm not sure what to do. I'm not sure what to do. I'm not sure what to do. I'm not sure what to do. I'm not sure what to do. I'm not sure what to do. I'm not sure what to do. I'm not sure what to do. I'm not sure what to do. I'm not sure what to do. I'm not sure what to do. I'm not sure what to do. I'm not sure what to do. I'm not sure what to do. I'm not sure what to do. I'm not sure what to do. I'm not sure what to do. I'm not sure what to do. I'm not sure what to do. I'm not sure what to do. I'm"}]


## LLaMA2 finetuned on NYC summit blogs

In [36]:
prompt = "Hi! I'm Aris and I am wondering what I should do today in sunny Athens."
print(f'{basic_endpoint_ft}: {query_endpoint({"inputs": prompt, "parameters": {"max_new_tokens": 200, "top_p": 0.9, "temperature": 0.01, "return_full_text": False}}, basic_endpoint_ft)}')

llama-2-13b-hf-nyc-finetuned: [{'generated_text': "\nI'm a software developer, a data scientist, a machine learning engineer, a researcher, a teacher, a speaker, a writer, a podcaster, a YouTuber, a blogger, a husband, a father, a friend, and a human being.\nI'm the co-founder and CEO of AI2Labs, a company that helps organizations harness the power of AI and ML to solve their most challenging problems.\nI'm also the co-founder of the Athens Machine Learning Meetup and the lead instructor for Amazon's Machine Learning and AI courses in Greek.\nI'm passionate about technology, education, and making a positive impact in the world.\nI'm always looking for new opportunities to learn, grow, and collaborate with others who share my passions.\nIf you'd like to connect, feel free to reach out to me on"}]


## LLaMA2-chat

In [38]:
prompt = "Hi! I'm Aris and I am wondering what I should do today in sunny Athens."
print(f'{chat_endpoint}: {query_endpoint({"inputs": [[{"role": "user", "content": prompt}]], "parameters": {"max_new_tokens": 200, "top_p": 0.9, "temperature": 0.01}}, chat_endpoint)}')

jumpstart-dft-meta-textgeneration-llama-2-13b-f: [{'generation': {'role': 'assistant', 'content': " Hello Aris! Sunny Athens is a beautiful place to explore, and there are plenty of things to do to make your day memorable. Here are some suggestions based on your interests:\n\n1. Visit the Acropolis: This ancient citadel sits atop a hill and is home to some of the most iconic landmarks of Athens, including the Parthenon, the Propylaea, and the Erechtheion. The Acropolis offers breathtaking views of the city, and you can spend hours exploring its history and architecture.\n2. Explore the National Archaeological Museum: If you're interested in history and artifacts, this museum is a must-visit. It houses an extensive collection of Greek antiquities, including the world-famous Mask of Agamemnon and the statue of the Athena Parthenos.\n3. Wander through the Monast"}}]


## LLaMA2-chat finetuned on NYC summit blogs

In [39]:
prompt = "Hi! I'm Aris and I am wondering what I should do today in sunny Athens."
print(f'{chat_endpoint_ft}: {query_endpoint({"inputs": json.dumps([[{"role": "user", "content": prompt}]]), "parameters": {"max_new_tokens": 200, "top_p": 0.9, "temperature": 0.01}}, chat_endpoint_ft)}')

llama-2-13b-chat-hf-nyc-finetuned: [{'generated_text': '\n\n[\n{"role": "google_assistant", "content": "Hello Aris! There are so many great things to do in Athens, it really depends on your interests. Would you like to learn more about ancient Greek history and culture? You could visit the Acropolis or the National Archaeological Museum. Or maybe you\'d prefer to explore the vibrant modern side of Athens? You could check out the Psirri neighborhood, known for its trendy bars and restaurants, or visit the Monastiraki Flea Market for some unique souvenirs. Whatever you choose, I\'m sure you\'ll have a great time in this amazing city!"}\n]\n\n[\n{"role": "user", "content": "Wow, there\'s so much to see and do! I think I\'d like to learn more about ancient Greek history and culture. Can you tell me more about the Ac'}]


In [40]:
prompt = "<s> [INST] Hi! I'm Aris and I am wondering what I should do today in sunny Athens. [/INST]"
print(f'{chat_endpoint_ft}: {query_endpoint({"inputs": prompt, "parameters": {"max_new_tokens": 200, "top_p": 0.9, "temperature": 0.01}}, chat_endpoint_ft)}')

llama-2-13b-chat-hf-nyc-finetuned: [{'generated_text': ' Hello Aris! Sunny Athens is a beautiful place to explore, and there are plenty of things to do to make your day memorable. Here are some suggestions based on your interests:\n\n1. Visit Acropolis: Start your day by visiting the iconic Acropolis, a UNESCO World Heritage Site and the most famous landmark of Athens. Explore the Parthenon, the Propylaea, the Erechtheion, and the Temple of Athena Nike.\n2. Wander in the Monastiraki Flea Market: After exploring the Acropolis, head to the Monastiraki Flea Market, located near the metro station of the same name. You can find everything from vintage clothing to antiques, souvenirs, and local snacks.\n3. Stroll through the National Garden: The National Garden is a peaceful oasis in the heart'}]


# Do the different models know what P5 instances are?
## LLaMA2-13b 

In [27]:
prompt = "Amazon EC2 P5 instances are equipped with GPUs of the type"
print(f'{basic_endpoint}: {query_endpoint({"inputs": prompt, "parameters": {"max_new_tokens": 200, "top_p": 0.9, "temperature": 0.01, "return_full_text": False}}, basic_endpoint)}')

jumpstart-dft-meta-textgeneration-llama-2-13b: [{'generation': ' NVIDIA Tesla V100.\nThe NVIDIA Tesla V100 is a GPU that is designed for high-performance computing. It is based on the NVIDIA Volta architecture and is equipped with 32 GB of HBM2 memory. The V100 is capable of delivering up to 100 teraflops of performance and is designed for use in data centers and supercomputers.\nThe NVIDIA Tesla V100 is a powerful GPU that is designed for high-performance computing. It is based on the NVIDIA Volta architecture and is equipped with 32 GB of HBM2 memory. The V100 is capable of delivering up to 100 teraflops of performance and is designed for use in data centers and supercomputers.\nThe NVIDIA Tesla V1'}]


## LLaMA2-13b finetuned on NYC summit blogs

In [29]:
prompt = "Amazon EC2 P5 instances are equipped with GPUs of the type"
print(f'{basic_endpoint_ft}: {query_endpoint({"inputs": prompt, "parameters": {"max_new_tokens": 200, "top_p": 0.9, "temperature": 0.01, "return_full_text": False}}, basic_endpoint_ft)}')

llama-2-13b-hf-nyc-finetuned: [{'generated_text': 'NVIDIA A100 for compute-intensive workloads, such as generative AI, high performance computing (HPC), and data analytics. P5 instances are ideal for training and running inference in models with trillions of parameters and for performing data analysis on vast datasets. They provide 8 times the GPU memory and 64 times the compute performance in floating point operations (FLOPs) compared to P4 instances.\nYou can use P5 instances in Amazon Elastic Container Service (ECS) with the most common container tools and frameworks, such as Docker, PyTorch, Tensorflow, Jupyter Notebook, R Studio, and Venv, to deploy your applications quickly. You can also use P5 instances via Amazon Elastic MapReduce (EMR) and Amazon Managed HPC (MHPC) to orchestrate jobs and clusters and to interact with P5 instances via a Jupyter Notebook or'}]


## LLaMA2-13b-chat

In [30]:
prompt = "What are Amazon EC2 P5 instances? Which kind of GPUs are they equipped with?"
print(f'{chat_endpoint}: {query_endpoint({"inputs": [[{"role": "user", "content": prompt}]], "parameters": {"max_new_tokens": 200, "top_p": 0.9, "temperature": 0.01}}, chat_endpoint)}')

jumpstart-dft-meta-textgeneration-llama-2-13b-f: [{'generation': {'role': 'assistant', 'content': " Amazon Elastic Compute Cloud (EC2) P5 instances are a type of instance that provides high-performance computing resources for applications that require intense computational power. These instances are equipped with NVIDIA Tesla V100 GPUs, which are designed for machine learning, scientific simulations, and other high-performance computing workloads.\n\nThe NVIDIA Tesla V100 GPUs in P5 instances are based on the Volta architecture and offer a number of features that make them well-suited for demanding workloads, including:\n\n* 5120 CUDA cores for high-performance computing\n* 640 tensor cores for accelerating deep learning and other matrix-based workloads\n* 16 GB of GDDR6 memory for fast data transfer and processing\n* Support for NVIDIA's GPU-accelerated libraries and frameworks, such as CU"}}]


## LLaMA2-13b-chat finetuned on NYC summit blogs

In [31]:
prompt = "What are Amazon EC2 P5 instances? Which kind of GPUs are they equipped with?"
print(f'{chat_endpoint_ft}: {query_endpoint({"inputs": json.dumps([[{"role": "user", "content": prompt}]]), "parameters": {"max_new_tokens": 200, "top_p": 0.9, "temperature": 0.01, "return_full_text": False}}, chat_endpoint_ft)}')

llama-2-13b-chat-hf-nyc-finetuned: [{'generated_text': '\n\n{"response": "Amazon EC2 P5 instances are general-purpose instances that provide extreme performance for applications that require heavy graphics processing and complex machine learning (ML) models. They are powered by NVIDIA H100 Tensor Core GPUs and third- generation AMD EPYC processors. The H100 GPUs provide 640 GB of high bandwidth GPU memory, enabling you to run your most demanding applications with massive datasets in real time. P5 instances also provide 2000 GB of high-speed CPU memory, allowing you to process vast amounts of data in memory without having to access disk storage. This reduces the processing time and improves response time. You can use these instances for applications such as computer vision, video encoding, genome analysis, and language model training."}\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'}]


In [41]:
prompt = "<s> [INST] What are Amazon EC2 P5 instances? Which kind of GPUs are they equipped with? [/INST]"
print(f'{chat_endpoint_ft}: {query_endpoint({"inputs": prompt, "parameters": {"max_new_tokens": 200, "top_p": 0.9, "temperature": 0.01, "return_full_text": False}}, chat_endpoint_ft)}')

llama-2-13b-chat-hf-nyc-finetuned: [{'generated_text': ' Amazon Elastic Compute Cloud (Amazon EC2) P5 instances are a type of compute-optimized instance that provides increased performance for compute-intensive workloads, particularly those that can benefit from high levels of GPU acceleration. These instances are designed for applications that require advanced machine learning (ML), high performance computing (HPC), and graphics processing.\n\nP5 instances are equipped with NVIDIA H100 Tensor Core GPUs, which provide fast performance for training and running inference for ML models. These GPUs also provide 400 GB/s of memory bandwidth and 320 TFLOPS of floating point performance for HPC applications.\n\nThe key features of P5 instances are:\n\n* Processor: Third-generation AMD EPYC processors (with 7nm CPU and 32C16G1D (384 cores) or 32C2'}]


In [ ]:
prompt = """
Blogpost conclusion: 
In conclusion, this blog post delves into the critical process of infusing domain-specific knowledge into large language models (LLMs) like LLaMA2, emphasizing the importance of addressing challenges related to helpfulness, honesty, and harmlessness when designing LLM-powered applications for enterprise-grade quality. The primary focus here is on the parametric approach to fine-tuning, which efficiently injects niche expertise into foundation models without compromising their general linguistic capabilities.The blog highlights the steps involved in fine-tuning LLaMA2 using parameter-efficient fine-tuning techniques, such as the qLoRA approach, and how this process can be conducted on Amazon SageMaker. By adopting this approach, practitioners can adapt LLaMA2 to specific domains, ensuring that the models remain up-to-date with recent knowledge even beyond their original training data. The article also underscores the versatility of this approach, showing that it can be applied to models like LLaMA2-chat, which have already undergone task-specific fine-tuning. This opens up opportunities to infuse knowledge into LLMs without the need for extensive instruction or chat-based fine-tuning, preserving their task-specific nature.
Task: 
Please extract the main takeaways from this blogpost.
"""

print(f'{chat_endpoint}: {query_endpoint({"inputs": [[{"role": "user", "content": prompt}]], "parameters": {"max_new_tokens": 200, "top_p": 0.9, "temperature": 0.01}}, chat_endpoint)}')